# Exercise 2


_course: quantum cryptography for beginners
<br>date: 1 september 2020
<br>author: burton rosenberg_

In this exercise we look at the Toffoli gate, and the use of ancillaries. 

In the second part, we begin investigating the quantum one time pad.


## Reversibility

The state of a register of qubits is a vector in a very high dimensional complex space.

- By complex we mean the numbers that describe the state are complex numbers. 
  To be completely clear, complex numbers include the reals, but also the number i, 
  sometimes called j when confusion with i would occur, that is one of the square roots of -1.
  The other square root of -1 is -i.
- By high dimension, we mean dimension 2<sup>n</sup> for n qubits.

Quantum gates then evolve the state vector according to the rules of quantum physics, and 
in particular, each vector evolves along a path in the complex space so that no paths intersect
or merge. The evolution is timed and stopped when the gate has had its intended effect.

Since these paths do not merger or intersect, it is also possible to evolve the state back to
where it came along the very path that carried it forward. 

Therefore &mdash; _every quantum circuit must be reversable!

Except when measured. When measured the quantum state becomes classical and we learn only that
the state measured as among the possible states, according to the measurement, and we destroy
any possibility of learning what other states were possible in a measurement. It is absolutely 
the opposite of reversable.

These two situations are completely the opposite of the classical case. In the classical case
our operations are often irreversable.

**Example:** Consider the following circuit, with two classical input, two classical outputs, and a 
single logical and gate.

<pre>
          +----+ 
 ---------|     \
          | AND  |-------
      +---|     /
      |   +----+ 
 -----+------------------
 
</pre>

Is this circuit reversable? I leave it to you to prove it is not. The and gate looses information
even, so it is impossible just looking at the two output bits to know what are the two input bits.

However, for classical measurement, there is not lose of information and no uncertainty about what
the state was before the measurement. If the two bits output bits are, say 0 and 1, then that is
exactly what they were before the measurement. We are actually measuring something that is there,
not sampling a probability distribution.

### CCX: the Toffoli gate

In order to have quantum version of a logical-and gate, we have to embed the operation in a 
greater, reversible function. Not only must it be reversible, but, due to the laws of quantum
physics, we cannot copy, clone, or destroy our qubits. 

The solution is to create a three qubit circuit, where the first two qubits pass unchanged to the
output, but they control whether to negate or not the third qubit. This is like the CNOT, but 
with two controls, not one, and the inversion occurs if both controls are one; else no inversion 
occurs. 

If the third bit is set to $|0\rangle$, then it remains $|0\rangle$ unless the two controls are 
$|1\rangle$, and in that case, the third qubit becomes a $|1\rangle$. Hence this circuit 
calculates a logical-and in the case that a third qubit is introduced, and initialized to zero.

The need to introduce additional, zeroed qubits is necessary for quantum computation to abide
by the reversibility. These qubits are called _ancillary_ bits for the computation.

This double controlled not gate called the Toffoli Gate, named after Tommaso Toffoli, who invented
it while at MIT and exploring the deeper meaning of computation.



In [1]:

import qiskit
import time, math

from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.providers.jobstatus import JOB_FINAL_STATES, JobStatus
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer

qiskit.__qiskit_version__


args_g = []

# your api token from IBM, first time run.
# after that None is good

#api_token = 'abcdefghijklmnopqrstuvwxyz'
api_token = None 

def load_or_save_IBMQ_account(api_token=None):
    global args_g
    if api_token:
        # only needs to be done once
        # then is stored in e.g. ~/.qistkit/qiskitrc
        IBMQ.save_account(api_token)
    provider = IBMQ.load_account()
    return provider

def list_backends(provider):
    global args_g
    backends = provider.backends()
    print('backends available:')
    for be in backends:
        st = be.status()
        if st.operational:
            print(f'\t{be.name()}, pending jobs:{st.pending_jobs}')

            
def run_quantum_circuit_on_backend(quantum_circuit,provider,backend):
    backend = provider.get_backend(backend)
    qobj = assemble(transpile(quantum_circuit, backend=backend), backend=backend)
    job = backend.run(qobj)
    return job


def wait_for_job(backend, job, wait_interval=5):
    backend = provider.get_backend(backend)
    retrieved_job = backend.retrieve_job(job.job_id())
    start_time = time.time()
    job_status = job.status()
    while job_status not in JOB_FINAL_STATES:
        print(f'Status @ {time.time() - start_time:0.0f} s: {job_status.name},'
              f' est. queue position: {job.queue_position()}')
        time.sleep(wait_interval)
        job_status = job.status()


provider = load_or_save_IBMQ_account(api_token)
list_backends(provider)



/Users/ojo/anaconda3/lib/python3.6/site-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


backends available:
	ibmq_qasm_simulator, pending jobs:1
	ibmqx2, pending jobs:6
	ibmq_16_melbourne, pending jobs:3
	ibmq_vigo, pending jobs:29
	ibmq_ourense, pending jobs:21
	ibmq_valencia, pending jobs:51
	ibmq_london, pending jobs:10
	ibmq_burlington, pending jobs:7
	ibmq_essex, pending jobs:9
	ibmq_armonk, pending jobs:0
	ibmq_santiago, pending jobs:3


In [2]:
# choose your backend

backend = 'ibmq_qasm_simulator'
#backend = 'ibmq_armonk'
#backend = 'ibmq_vigo'
#backend = 'ibmq_london'

# and so forth ... chose from the results given by provider.backends()

In [3]:
def make_qubit_triple():
    # make the circuit
    q = QuantumRegister(3)
    c = ClassicalRegister(3)
    return (QuantumCircuit(q, c),q,c)

def add_measurement_qubit_triple(qubit_triple):
    qubit_triple[0].measure(qubit_triple[1],qubit_triple[2])
    print('\n-------- CIRCUIT ---------')
    print(qubit_triple[0].draw(output='text'))
    print('-------------------------\n')
    return qubit_triple

def add_value_qubit_triple(qt, value):

    if value%2 == 1 :
        qt[0].x(0)
    if value%4>1:
        qt[0].x(1)
    if value>3:
        qt[0].x(2)
    
    qt[0].barrier()
    return qt

def add_toffoli_qubit_triple(qt):
    qt[0].ccx(0,1,2)


def build_toffoli_test(value):
    qt = make_qubit_triple()
    add_value_qubit_triple(qt,value)
    add_toffoli_qubit_triple(qt)
    add_measurement_qubit_triple(qt)
    return qt

build_toffoli_test(0)
build_toffoli_test(7)


-------- CIRCUIT ---------
       ░      ┌─┐      
q0_0: ─░───■──┤M├──────
       ░   │  └╥┘┌─┐   
q0_1: ─░───■───╫─┤M├───
       ░ ┌─┴─┐ ║ └╥┘┌─┐
q0_2: ─░─┤ X ├─╫──╫─┤M├
       ░ └───┘ ║  ║ └╥┘
c0: 3/═════════╩══╩══╩═
               0  1  2 
-------------------------


-------- CIRCUIT ---------
      ┌───┐ ░      ┌─┐      
q1_0: ┤ X ├─░───■──┤M├──────
      ├───┤ ░   │  └╥┘┌─┐   
q1_1: ┤ X ├─░───■───╫─┤M├───
      ├───┤ ░ ┌─┴─┐ ║ └╥┘┌─┐
q1_2: ┤ X ├─░─┤ X ├─╫──╫─┤M├
      └───┘ ░ └───┘ ║  ║ └╥┘
c1: 3/══════════════╩══╩══╩═
                    0  1  2 
-------------------------



(<qiskit.circuit.quantumcircuit.QuantumCircuit at 0x7fd5ab7ea5f8>,
 QuantumRegister(3, 'q1'),
 ClassicalRegister(3, 'c1'))

In [4]:
# we use this circuit only on the inputs 0-3. The 3ird bit just collects the output.
# however it does calculate on inputs 4-7, and is an extension of logical-and that makes
# the circuit reversible.

for i in range(4):
    tt = build_toffoli_test(i)
    job = run_quantum_circuit_on_backend(tt[0],provider,backend)
    print(f'results: waiting for results from backend {backend} ...')
    wait_for_job(backend, job)
    result = job.result()
    print(f'results: {result.get_counts()}')


-------- CIRCUIT ---------
       ░      ┌─┐      
q2_0: ─░───■──┤M├──────
       ░   │  └╥┘┌─┐   
q2_1: ─░───■───╫─┤M├───
       ░ ┌─┴─┐ ║ └╥┘┌─┐
q2_2: ─░─┤ X ├─╫──╫─┤M├
       ░ └───┘ ║  ║ └╥┘
c2: 3/═════════╩══╩══╩═
               0  1  2 
-------------------------

results: waiting for results from backend ibmq_qasm_simulator ...
Status @ 0 s: VALIDATING, est. queue position: None
results: {'000': 1024}

-------- CIRCUIT ---------
      ┌───┐ ░      ┌─┐      
q6_0: ┤ X ├─░───■──┤M├──────
      └───┘ ░   │  └╥┘┌─┐   
q6_1: ──────░───■───╫─┤M├───
            ░ ┌─┴─┐ ║ └╥┘┌─┐
q6_2: ──────░─┤ X ├─╫──╫─┤M├
            ░ └───┘ ║  ║ └╥┘
c3: 3/══════════════╩══╩══╩═
                    0  1  2 
-------------------------

results: waiting for results from backend ibmq_qasm_simulator ...
Status @ 0 s: VALIDATING, est. queue position: None
results: {'001': 1024}

-------- CIRCUIT ---------
             ░      ┌─┐      
q11_0: ──────░───■──┤M├──────
       ┌───┐ ░   │  └╥┘┌─┐   
q11_1: ┤ X ├─

## Exercise A

Classical computers can add numbers. We would like to look at the analog of this circuit for a 
quantum computer. 

A classical binary computer adds numbers when they are given in a binary number system of one's and
zero's. Although there is a large number of techniques to improve the speed of addition, we will consider
the simple addition algorithm the works bit by bit, from least significant bit to most significant bit.

Each bit addition follows this logic. Suppose the numbers $A$ and $B$ are added, and we are considering just the
bit $a$ and $b$ from $A$ and $B$, respectively. There is possibily a carry in $c_i$ carried over from the
just previous bit addition.

We need a sum bit $s$ and an carry out bit $c_o$. 

Write down the formula for $s$ and $c_o$ as a function of $a, b$ and $c_i$. The formula is to be
in logic: use 
- $\land$, logical and,
- $\lor$, logical or, 
- $\lnot$, logical not, and
- $\oplus$, exclusive or. 

Hint: If we did not worry about carries we have the formula,

$$
s = a \oplus b
$$

In [5]:
def s(a,b,c_in):
    # write code to replace the following line
    return False

def c_out(a,b,c_in):
    # write code to replace the following line
    return False

test_vector = [(0,0,0,0,0),(1,0,0,1,0),(0,1,0,1,0),(1,1,0,0,1),
               (0,0,1,1,0),(1,0,1,0,1),(0,1,1,0,1),(1,1,1,1,1)]

def test_classical_formula():
    for test in test_vector:
        (a,b,c_in,s_correct,c_out_correct) = [bool(test[i]) for i in range(5)]
        if s(a,b,c_in)!=s_correct:
            print('failed the test')
            return
        if c_out(a,b,c_in)!=c_out_correct:
            print('failed the test')
            return
    print('passed the test')

test_classical_formula()

failed the test



## Exercise B

Rewrite the equations for a five qubit circuit, with the first three qubits $a$, $b$ and $c_{in}$ and the 
fourth and fifth qubits $s$ and $c_{out}$ (respectively). The output passes the first three qubits
unchange; the fourth and fifth qubits are ancillaries, set to zero on input and have values $s$
and $c_{out}$ as output.

Make a circuit and test it on all possible 0 and 1 states for the first three qubits.



In [6]:

def make_qubit_n(n):
    # make the circuit
    q = QuantumRegister(n)
    c = ClassicalRegister(n)
    return (QuantumCircuit(q, c),q,c)

def add_measurement_qubit_n(q,n):
    q[0].barrier()
    q[0].measure(q[1],q[2])
    print('\n-------- CIRCUIT ---------')
    print(q[0].draw(output='text'))
    print('-------------------------\n')
    return q

def add_small_value_qubit_n(q, n, value):

    if value%2 == 1 :
        q[0].x(0)
    if value%4>1:
        q[0].x(1)
    if value>3:
        q[0].x(2)
    
    q[0].barrier()
    return q



In [7]:
def add_adder_5(q):
    
    #
    # write code to build your adder
    #
    
    return q

def build_adder_5(value):
    q = make_qubit_n(5)
    add_small_value_qubit_n(q,5,value)
    add_adder_5(q)
    add_measurement_qubit_n(q,5)
    return q

def test_adder_5():
    for value in range(8):
        q_adder= build_adder_5(value)
        job = run_quantum_circuit_on_backend(q_adder[0],provider,backend)
        print(f'results: waiting for results from backend {backend} ...')
        wait_for_job(backend, job)
        result = job.result()
        print(f'results: {result.get_counts()}')
        
test_adder_5()


-------- CIRCUIT ---------
        ░  ░ ┌─┐            
q21_0: ─░──░─┤M├────────────
        ░  ░ └╥┘┌─┐         
q21_1: ─░──░──╫─┤M├─────────
        ░  ░  ║ └╥┘┌─┐      
q21_2: ─░──░──╫──╫─┤M├──────
        ░  ░  ║  ║ └╥┘┌─┐   
q21_3: ─░──░──╫──╫──╫─┤M├───
        ░  ░  ║  ║  ║ └╥┘┌─┐
q21_4: ─░──░──╫──╫──╫──╫─┤M├
        ░  ░  ║  ║  ║  ║ └╥┘
 c6: 5/═══════╩══╩══╩══╩══╩═
              0  1  2  3  4 
-------------------------

results: waiting for results from backend ibmq_qasm_simulator ...
Status @ 0 s: VALIDATING, est. queue position: None
results: {'00000': 1024}

-------- CIRCUIT ---------
       ┌───┐ ░  ░ ┌─┐            
q24_0: ┤ X ├─░──░─┤M├────────────
       └───┘ ░  ░ └╥┘┌─┐         
q24_1: ──────░──░──╫─┤M├─────────
             ░  ░  ║ └╥┘┌─┐      
q24_2: ──────░──░──╫──╫─┤M├──────
             ░  ░  ║  ║ └╥┘┌─┐   
q24_3: ──────░──░──╫──╫──╫─┤M├───
             ░  ░  ║  ║  ║ └╥┘┌─┐
q24_4: ──────░──░──╫──╫──╫──╫─┤M├
             ░  ░  ║  ║  ║  ║ └╥┘
 c7: 5/════════════╩══╩


## Exercise C

Reduce the adder to a single ancillary. The $c_{in}$ qubit is the $s$ output qubit; the $c_{out}$ qubit
is an ancillary.


In [8]:
def add_adder_4(q):
    
    #
    # write code to build your adder
    # 

    return q

def build_adder_4(value):
    q = make_qubit_n(4)
    add_small_value_qubit_n(q,4,value)
    add_adder_4(q)
    add_measurement_qubit_n(q,4)
    return q

def test_adder_4():
    for value in range(8):
        q_adder= build_adder_4(value)
        job = run_quantum_circuit_on_backend(q_adder[0],provider,backend)
        print(f'results: waiting for results from backend {backend} ...')
        wait_for_job(backend, job)
        result = job.result()
        print(f'results: carry in is {value>3}, output: {result.get_counts()}')
        
test_adder_4()


-------- CIRCUIT ---------
        ░  ░ ┌─┐         
q52_0: ─░──░─┤M├─────────
        ░  ░ └╥┘┌─┐      
q52_1: ─░──░──╫─┤M├──────
        ░  ░  ║ └╥┘┌─┐   
q52_2: ─░──░──╫──╫─┤M├───
        ░  ░  ║  ║ └╥┘┌─┐
q52_3: ─░──░──╫──╫──╫─┤M├
        ░  ░  ║  ║  ║ └╥┘
c14: 4/═══════╩══╩══╩══╩═
              0  1  2  3 
-------------------------

results: waiting for results from backend ibmq_qasm_simulator ...
Status @ 0 s: VALIDATING, est. queue position: None
results: carry in is False, output: {'0000': 1024}

-------- CIRCUIT ---------
       ┌───┐ ░  ░ ┌─┐         
q55_0: ┤ X ├─░──░─┤M├─────────
       └───┘ ░  ░ └╥┘┌─┐      
q55_1: ──────░──░──╫─┤M├──────
             ░  ░  ║ └╥┘┌─┐   
q55_2: ──────░──░──╫──╫─┤M├───
             ░  ░  ║  ║ └╥┘┌─┐
q55_3: ──────░──░──╫──╫──╫─┤M├
             ░  ░  ║  ║  ║ └╥┘
c15: 4/════════════╩══╩══╩══╩═
                   0  1  2  3 
-------------------------

results: waiting for results from backend ibmq_qasm_simulator ...
Status @ 0 s: VALIDATING, es


## Classical One-time Pad

There is a perfectly unbreakable cipher. We will present this cipher in this exercise, and 
present the framework by which the cipher is judged unbreakable. You will then have an opportunity
to try to break it. Hopefully this will give you an intuitation about its unbreakability.

The cipher is called the Vernam Cipher, or the One Time Pad. It is a very old cipher, described
by AT&T researcher Gilbert Vernam in 1919. While unbreakable, it has severe, practical drawbacks:

- the key must be made of completely unbaised coin flips
- the key must be made of as many coin flips as there are bits in the message
- the key must be discarded after use; no part of the key can be reused.

In practice, a long random sequence of coin flips is created and recorded, with a copy going to 
each legitimate participant in the channel. Say this is Alice and Bob. They both get a copy of the 
random bits. This is called the key.

Alice selects a message that she wishes Bob to know. This is called the plaintext, and is a random,
but not completely random, message. If could be, for instance, letter strings that is a readable 
text in english. 

She creates the ciphertext by combining the plaintext and key. She does this by encoding the 
text as a random string (using ASCII binary codes for english characters, for example), and 
bitwise exclusive or's the plaintext with the key. 

That is, for each plaintext bit, if the corresponding key bit is 0, Alice just copies that plaintext
bit to the corresponding ciphertext bit; if the key bit is 1, Alice complements the plaintext bit
before copying it to the ciphertext.

Bob perfectly recovers the plaintext from the ciphertext by repeating what Alice did. He will 
complement when Alice complemented, and will pass the bit 
unchanged when Alice passed the bit unchanged. In either case, the plaintext bit is recovered,
and bit  by bit the plaintext in its entirety is recovered.



In [9]:
import random
import math

class OTP:
    def __init__(self,n):
        self.n = n
        self.key = []

    def generate_key(self):
        self.key = [random.randint(0,1) for i in range(self.n)]
        return self.key

    def generate_plaintext(self,prob_of_zero):
        return [int(random.random()>prob_of_zero) for i in range(self.n)]

    def encrypt(self,msg):
        assert(len(msg)==self.n)
        return [int(msg[i]!=self.key[i]) for i in range(self.n)]

    def aux_statistic(self,msg):
        ones = 0
        for i in range(len(msg)):
            if msg[i]==1:
                ones += 1
        return ones/len(msg)

    def aux_test(self,prob_of_zero,trials=256):
        for t in range(trials):
            self.generate_key()
            msg = self.generate_plaintext(prob_of_zero)
            enc = self.encrypt(msg)
            dec = self.encrypt(enc)
            
            for i in range(self.n):
                assert(dec[i]==msg[i])
                f = self.aux_statistic(enc)
                assert(math.isclose(f,0.5,abs_tol=0.2))

        return True



In [10]:

def alice(otp,plaintext):
    # given an otp object and a plaintext, have alice:
    # - encrypt the plaintext
    # - return the ciphertext
    
    # write code to replace the following line
    return [0]*otp.n
    

def bob(otp,ciphertext):
    # given an otp object and a ciphertext, have bob
    # - decrypt the ciphertext
    # - return the plain text
    
    # write code to replace the following line
    return [1]*otp.n
    

def otp_test(p=0.5,n=1024):
    otp = OTP(n)
    otp.generate_key()
    
    # replace this line with a call to generate_plaintext
    msg = [0]*otp.n
    
    
    # demonstrate it works
    enc = alice(otp,msg)
    dec = bob(otp,enc)
    
    for i in range(n):
        if msg[i]!=dec[i]:
            return False
    return True

otp_test()

False


## The adversary game

The security is given by this game. Alice and Bob act as above, and the adversary is Eve, who 
knows the way in which plaintexts are chosen, although not exactly which is chosen, and she witnesses
the ciphertext - &mdash; Eve is an eavesdropper.

With all this information and an infinity of computation, Eve guesses the plaintext.
Her guess and the plaintext are compared to see what percentage of the message she guessed correctly.

She wins the game if she has any more guesses correct than she is entitled to. 

What is Eve entitled to? If Alice is most likely to choose a certain plaintext, Eve is entitled to know that, 
and can guess that as the plaintext. Eve can do this without even looking at the ciphertext.
We give Eve the probability information about the plaintext choice, so that she might take advantage of 
that in her strategy.

 


## Exercise D


The following code gives an Eve object and an AdversarialGame object. You are to find strategies for
Eve to win the adversarial game. 

Hint: The given strategy is for Eve to just flip coins. This will give her a probability of being correct half the time. This is considered zero advantage. Other strategies that might be better is for Eve to flip a coin based on  Alice's coin bias, or by looking at the ciphertext for hints.



In [11]:
class Eve:
    
    def __init__(self,n):
        self.n = n
     
    def attack(self,enc,prob_of_zero):
  
        #
        # implement attack strategies to maximize advantage
        #

        return [random.randint(0,1) for i in range(len(enc))]
    
       


In [12]:
class AdversarialGame:
    
    def __init__(self,n=1024):
        self.n = n
        self.otp = OTP(n)
            
    def adversary_game(self,prob_of_zero):
        self.otp.generate_key()
        msg = self.otp.generate_plaintext(prob_of_zero)
        enc = self.otp.encrypt(msg)
        
        eve = Eve(self.n)
        eve_guess = eve.attack(enc,prob_of_zero)
        
        correct = 0
        for i in range(self.n):
            if eve_guess[i]==msg[i]:
                correct += 1
        return correct/self.n

    def run_game(self):
        max_correct = 0.0
        avg_correct = 0.0
        print(f'prob\tcorrect')
        print(f'-------------------------')
        for p_int in range(11):
            p = p_int/10.0
            correct = self.adversary_game(p)
            if correct > max_correct:
                max_correct = correct
            avg_correct = avg_correct + correct
            print(f'{p}\t{correct:.4f}')
            
        avg_correct = avg_correct/11.0
        advantage = (avg_correct-0.5)*4.0
        if advantage<0.0: advantage = 0.0
        if advantage>1.0: advantage = 1.0

        print(f'\nmax corect:\t{max_correct:.4f}\navg correct:\t{avg_correct:.4f}')
        print(f'advantage:\t{advantage:.4f}')
        
        if (math.isclose(advantage,1.0,abs_tol=0.02)):
            print(f'\n*** YOU WIN! ***')


AdversarialGame(1024).run_game()



prob	correct
-------------------------
0.0	0.4922
0.1	0.5146
0.2	0.4756
0.3	0.5420
0.4	0.4902
0.5	0.5088
0.6	0.4766
0.7	0.5020
0.8	0.5146
0.9	0.5049
1.0	0.5068

max corect:	0.5420
avg correct:	0.5026
advantage:	0.0103
